# 1. Import the necessary libraries for you to be able to query 4square API

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass
import os
import time
from pymongo import MongoClient
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
token = os.getenv("token")

In [3]:
def gets_coordinates_from_string(where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"
    res = requests.get(url_geocode)
    try:
        return res.json()
    except:
        time.sleep(2)
        gets_coordinates_from_string (where)

In [13]:
def requests_for_foursquare (query, lat, lon, radius=None, limit=1):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&limit={limit}"
    if radius:
        url += f'&radius={radius}'

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [14]:
iron_lat = '41.3977461'
iron_long = '2.1876744'
# res = gets_coordinates_from_string ("Carrer Pamplona 96")
# res

res = requests_for_foursquare("coffee", iron_lat, iron_long, limit=10)
res = res['results']
res[0]

{'fsq_id': '5b8521d489e490002cefef18',
 'categories': [{'id': 13035,
   'name': 'Coffee Shop',
   'short_name': 'Coffee Shop',
   'plural_name': 'Coffee Shops',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
    'suffix': '.png'}},
  {'id': 13065,
   'name': 'Restaurant',
   'short_name': 'Restaurant',
   'plural_name': 'Restaurants',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'VeryLikelyOpen',
 'distance': 354,
 'geocodes': {'main': {'latitude': 41.397168, 'longitude': 2.183123},
  'roof': {'latitude': 41.397168, 'longitude': 2.183123}},
 'link': '/v3/places/5b8521d489e490002cefef18',
 'location': {'address': 'Ausiàs Marc, 151',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'formatted_address': 'Ausiàs Marc, 151, 08013 Barcelona Catalunya',
  'locality': 'Barcelona',
  'postcode': '08013',
  'region': 'Catalunya'},
 'name': 'Three Marks Coffee',
 'related

In [25]:
def get_name_location(rest):
    return {'name': rest['name'], 'location': rest['location'], 'latitude': rest['geocodes']['main']['latitude'], 'longitude': rest['geocodes']['main']['longitude']}

coffes = [get_name_location(i) for i in res]
coffes[0:2]

[{'name': 'Three Marks Coffee',
  'location': {'address': 'Ausiàs Marc, 151',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'formatted_address': 'Ausiàs Marc, 151, 08013 Barcelona Catalunya',
   'locality': 'Barcelona',
   'postcode': '08013',
   'region': 'Catalunya'},
  'latitude': 41.397168,
  'longitude': 2.183123},
 {'name': "Nømad Roaster's Home",
  'location': {'address': 'Calle de Pujades, 95',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'cross_street': 'C. de Badajoz',
   'formatted_address': 'Calle de Pujades, 95 (C. de Badajoz), 08005 Barcelona',
   'locality': 'Barcelona',
   'postcode': '08005'},
  'latitude': 41.397977,
  'longitude': 2.195589}]

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [20]:
res = requests_for_foursquare("coffee", iron_lat, iron_long, limit=5)
res = res['results']
coffes = [get_name_location(i) for i in res]
coffes


[{'name': 'Three Marks Coffee',
  'location': {'address': 'Ausiàs Marc, 151',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'formatted_address': 'Ausiàs Marc, 151, 08013 Barcelona Catalunya',
   'locality': 'Barcelona',
   'postcode': '08013',
   'region': 'Catalunya'},
  'latitude': 41.397168,
  'longitude': 2.183123},
 {'name': "Nømad Roaster's Home",
  'location': {'address': 'Calle de Pujades, 95',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'cross_street': 'C. de Badajoz',
   'formatted_address': 'Calle de Pujades, 95 (C. de Badajoz), 08005 Barcelona',
   'locality': 'Barcelona',
   'postcode': '08005'},
  'latitude': 41.397977,
  'longitude': 2.195589},
 {'name': 'Brew Coffee',
  'location': {'address': 'Calle de Roger de Flor, 102',
   'admin_region': 'Cataluña',
   'country': 'ES',
   'cross_street': '',
   'formatted_address': 'Calle de Roger de Flor, 102, 08013 Barcelona Catalunya',
   'locality': 'Barcelona',
   'postcode': '08013',
   'region': 'Catalunya'

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [17]:
def foursquare_places(venue):
    endpoint = f'https://api.foursquare.com/v3/places/search?query={venue}'
    
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        response = requests.get(endpoint, headers=headers)
        response.raise_for_status()  # Check for any errors in the response
        data = response.json()
        
        data = data['results']
        results = []
        
        for i in data:
            name = i['name']
            location = i['geocodes']['main']
            lat = location.get('latitude')
            lng = location.get('longitude')
            results.append({
                'name': name,
                'lat': lat,
                'lng': lng
            })
        
        return results
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return []

In [33]:
res = foursquare_places("coffee")
res

[{'name': 'Dalston Coffee', 'lat': 41.384525, 'lng': 2.168502},
 {'name': 'Barcentral', 'lat': 41.38368, 'lng': 2.168826},
 {'name': 'Chok - The Chocolate Kitchen', 'lat': 41.382801, 'lng': 2.171393},
 {'name': 'Nømad Coffee Lab & Shop', 'lat': 41.388989, 'lng': 2.177167},
 {'name': 'Caravelle', 'lat': 41.382226, 'lng': 2.168842},
 {'name': 'Departure Coffee Co', 'lat': 41.38361, 'lng': 2.164875},
 {'name': 'Funky Bakers', 'lat': 41.394916, 'lng': 2.172866},
 {'name': "Satan's Coffee Corner", 'lat': 41.38264, 'lng': 2.175303},
 {'name': 'Roast Club', 'lat': 41.387825, 'lng': 2.157221},
 {'name': 'Cafés el Magnífico', 'lat': 41.383541, 'lng': 2.180957}]

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [34]:
pd.DataFrame(res).to_json('data/coffe_shops.json')

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [35]:
client = MongoClient("localhost:27017")
db = client["ironhack"]
collection = db['ironcoffee']
for coffe in coffes:
    collection.insert_one(coffe)